# Compare the parallel results to the serial case

In the previous notebook you have generated and executed 1 or, more likely, multiple parallel simulations. This notebook can be used to compare the results to the serial run to make sure that the parallel simulations were successful before continuing to study their performance characteristics.

### Imports

In [ ]:
# General
import numpy as np
import matplotlib.pyplot as plt
import pathlib as pl

# FloPy
from flopy.mf6.utils import Mf6Splitter

# Local
from utilities import *

## Parallel simulations

First list all available simulations using the helper function:

In [ ]:
ws = get_all_workspaces()

# print inventory
print(f"ndomains, workspace")
for key, val in ws.items():
  print(f"{key}, {val}")

Set the number of domains to compare

In [ ]:
# set for comparing against serial (ndomains = 1) results
ndomains = 4

if ndomains not in ws:
  raise ValueError(f"Error: workspace not available for ndomains = {ndomains}")

# set workspaces to compare (parallel versus serial)
serial_ws = get_serial_workspace()
parallel_ws = get_workspace(ndomains)

## Load the serial (single model) basin simulation

In [ ]:
# load the serial reference from disk
base_sim = flopy.mf6.MFSimulation.load(
    sim_ws=serial_ws,
    verbosity_level=0,
)

## Load the parallel basin simulation

In [ ]:
par_sim = flopy.mf6.MFSimulation.load(
    sim_ws=parallel_ws,
    verbosity_level=0,
)

## Comparison between parallel and serial results

Here the head values for the parallel, multi-model simulation are recombined using the Model Splitter and the JSON mapping file that was generated in the previous notebook. Then they can be compared to the results from the serial (base) simulation. For a successful parallel run, the maximum difference between the two should be of the order of the solver tolerance (NB compare the difference plot below with the value for `outer_dvclose` in the first notebook).

Start with pulling out the head values for the base simulation:

In [ ]:
# there is only one model here
base_gwf = base_sim.get_model()
times = base_gwf.output.head().get_times()

# get heads for last timestep
base_head = np.squeeze(base_gwf.output.head().get_data(totim=times[-1]))
hmin, hmax = (
    base_head.min(),
    np.where(base_head < 1e30, base_head, 0).max(),
)

# for plotting
contours = np.arange(0, 100, 10)

### Reconstruct parallel heads into a single head array

Load the node mapping array for the parallel basin model

In [ ]:
json_path = parallel_ws / "mfsplit_node_mapping.json"
mfsplit = Mf6Splitter(base_sim)
mfsplit.load_node_mapping(par_sim, json_path)

Build a dictionary with the model heads for each partition

In [ ]:
# get model names, equals nr. of domains
model_names = list(par_sim.model_names)

# create a dictionary with head arrays for the reconstruction
head_dict = {}
for mname in model_names:
    mnum = int(mname.split("_")[-1])
    head = par_sim.get_model(mname).output.head().get_data(totim=times[-1])
    head_dict[mnum] = head

And then reconstruct the head array using the functionality in the Model Splitter

In [ ]:
par_head = mfsplit.reconstruct_array(head_dict)

### Plot the results

Now we have head arrays for both simulations (parallel and serial) that we can compare. The last step is to plot them and their difference:

In [ ]:
# create the 3 head arrays for plotting
diff_head = par_head - base_head
head_array = [par_head, base_head, diff_head]

# and create a figure with three plots
fig = plt.figure(figsize=(6, 9))
titles = [f"parallel ({ndomains} partitions)", "serial", "difference (parallel - serial)"]
for idx in range(3):
    ax = fig.add_subplot(3, 1, idx + 1)
    ax.set_aspect("equal")
    ax.set_title(titles[idx])

    if idx < 2:
        levels = contours
        vmin = hmin
        vmax = hmax
    else:
        # the third plot is the difference and is pretty
        # much 0 everywhere, hence no contouring here
        levels = None
        vmin = None
        vmax = None

    pmv = flopy.plot.PlotMapView(model=base_gwf, ax=ax, layer=0)
    pa = pmv.plot_array(head_array[idx], vmin=vmin, vmax=vmax)
    if levels is not None:
        c = pmv.contour_array(
            head_array[idx],
            levels=levels,
            colors="white",
            linewidths=0.75,
            linestyles=":",
        )
        plt.clabel(c, fontsize=8)
    if base_gwf.modelgrid.idomain is not None:
        pmv.plot_inactive()
    plt.colorbar(pa, ax=ax, shrink=0.5)

plt.show()